In [61]:
import pandas as pd

In [ ]:
def get_tickers():
    df = pd.read_csv('sp500_joined_closes.csv')
    return df.columns[1:]
tickers = get_tickers()
hm_days = 7

In [89]:
ticker = 'AAPL'
df = pd.read_csv("stocks_dfs/{}.csv".format(ticker), index_col=0, parse_dates=['Date'])
df = df[['Adj Close','Volume']]
df.fillna(1, inplace=True)
for i in range(1, hm_days+1):
        df['{}d'.format(i)] = (df['Adj Close'].shift(-i) - df['Adj Close']) / df['Adj Close']

df['50MA'] = df['Adj Close'].rolling(50).mean()
df['25MA'] = df['Adj Close'].rolling(25).mean()
df['10MA'] = df['Adj Close'].rolling(10).mean()
df['5MA'] = df['Adj Close'].rolling(5).mean()

df['50STD'] = df['Adj Close'].rolling(50).std()
df['25STD'] = df['Adj Close'].rolling(25).std()
df['10STD'] = df['Adj Close'].rolling(10).std()

df['50UBB'] = df['50MA'] + (df['50STD'] * 2)
df['25UBB'] = df['25MA'] + (df['25STD'] * 2)

df['50LBB'] = df['50MA'] - (df['50STD'] * 2)
df['25LBB'] = df['25MA'] - (df['25STD'] * 2)
df.fillna(0, inplace=True)

In [90]:
df.head()

,Adj Close,Volume,1d,2d,3d,4d,5d,6d,7d,50MA,25MA,10MA,5MA,50STD,25STD,10STD,50UBB,25UBB,50LBB,25LBB
Date,,,,,,,,,,,,,,,,,,,,
2010-01-04,26.538483,123432400.0,0.001729,-0.014205,-0.016027,-0.009485,-0.018223,-0.029391,-0.015700,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-05,26.584366,150476200.0,-0.015906,-0.017725,-0.011195,-0.019918,-0.031066,-0.017399,-0.023089,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-06,26.161509,138040000.0,-0.001849,0.004787,-0.004076,-0.015405,-0.001517,-0.007299,-0.023890,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-07,26.113146,119282800.0,0.006648,-0.002232,-0.013582,0.000332,-0.005461,-0.022082,0.021180,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-08,26.286753,111902700.0,-0.008821,-0.020096,-0.006274,-0.012029,-0.028540,0.014435,-0.001179,0.0,0.0,0.0,26.336851,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.03
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

In [92]:
df['target'] = list(map( buy_sell_hold,
                                               df['1d'.format(ticker)],
                                               df['2d'.format(ticker)],
                                               df['3d'.format(ticker)],
                                               df['4d'.format(ticker)],
                                               df['5d'.format(ticker)],
                                               df['6d'.format(ticker)],
                                               df['7d'.format(ticker)] ))

In [93]:
df.head()

,Adj Close,Volume,1d,2d,3d,4d,5d,6d,7d,50MA,...,10MA,5MA,50STD,25STD,10STD,50UBB,25UBB,50LBB,25LBB,target
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,26.538483,123432400.0,0.001729,-0.014205,-0.016027,-0.009485,-0.018223,-0.029391,-0.015700,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2010-01-05,26.584366,150476200.0,-0.015906,-0.017725,-0.011195,-0.019918,-0.031066,-0.017399,-0.023089,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
2010-01-06,26.161509,138040000.0,-0.001849,0.004787,-0.004076,-0.015405,-0.001517,-0.007299,-0.023890,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2010-01-07,26.113146,119282800.0,0.006648,-0.002232,-0.013582,0.000332,-0.005461,-0.022082,0.021180,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2010-01-08,26.286753,111902700.0,-0.008821,-0.020096,-0.006274,-0.012029,-0.028540,0.014435,-0.001179,0.0,...,0.0,26.336851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [68]:
df['target'][0:4]



Date
2010-01-04    0
2010-01-05   -1
2010-01-06    0
2010-01-07    0
Name: target, dtype: int64

In [69]:
freq = 'D'
start_dataset = pd.Timestamp("2010-01-04 00:00:00", freq=freq)
end_training = pd.Timestamp("2018-12-31 00:00:00", freq=freq)

In [70]:
training_data = [
    {
        "start": str(start_dataset),
        "target": df['target'][start_dataset:end_training - 1].tolist()  # We use -1, because pandas indexing includes the upper bound 
    }
    
]
print(len(training_data))


1


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`


In [109]:
import pandas as pd
from collections import OrderedDict
import pytz

data = OrderedDict()
stock_data_preprocessed = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv("stocks_dfs/{}.csv".format(ticker), index_col=0, parse_dates=['Date'])
    df = df[['Adj Close','Volume']]
    df.fillna(1, inplace=True)
    for i in range(1, hm_days+1):
        df['{}d'.format(i)] = (df['Adj Close'].shift(-i) - df['Adj Close']) / df['Adj Close']
        
    df['50MA'] = df['Adj Close'].rolling(50).mean()
    df['25MA'] = df['Adj Close'].rolling(25).mean()
    df['10MA'] = df['Adj Close'].rolling(10).mean()
    df['5MA'] = df['Adj Close'].rolling(5).mean()

    df['50STD'] = df['Adj Close'].rolling(50).std()
    df['25STD'] = df['Adj Close'].rolling(25).std()
    df['10STD'] = df['Adj Close'].rolling(10).std()

    df['50UBB'] = df['50MA'] + (df['50STD'] * 2)
    df['25UBB'] = df['25MA'] + (df['25STD'] * 2)

    df['50LBB'] = df['50MA'] - (df['50STD'] * 2)
    df['25LBB'] = df['25MA'] - (df['25STD'] * 2)
    data[ticker] = df
    

stock_data_preprocessed = pd.concat(data.values(),keys=tickers,names=['Ticker','Date'])

stock_data_preprocessed = stock_data_preprocessed.swaplevel()
stock_data_preprocessed = stock_data_preprocessed.sort_index()
stock_data_preprocessed.to_csv('stock_data_preprocessed.csv')

In [112]:
A = stock_data_preprocessed.loc[(slice(None), slice('A')), :]
A.head()

,,Adj Close,Volume,1d,2d,3d,4d,5d,6d,7d,50MA,25MA,10MA,5MA,50STD,25STD,10STD,50UBB,25UBB,50LBB,25LBB
Date,Ticker,,,,,,,,,,,,,,,,,,,,
2010-01-04,A,20.436504,3815500.0,-0.010862,-0.014377,-0.015655,-0.015975,-0.015336,-0.027157,-0.019489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,A,20.214516,4186000.0,-0.003553,-0.004845,-0.005168,-0.004522,-0.016473,-0.008721,0.006137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,A,20.142689,3243700.0,-0.001297,-0.001621,-0.000972,-0.012966,-0.005187,0.009725,-0.013614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,A,20.116571,3095100.0,-0.000325,0.000325,-0.011685,-0.003895,0.011036,-0.012334,-0.000325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,A,20.110041,3733900.0,0.000649,-0.011364,-0.003572,0.011364,-0.012013,0.000000,-0.005520,NaN,NaN,NaN,20.204064,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.03
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

In [118]:
A['target'] = list(map( buy_sell_hold,
                                               A['1d'.format(ticker)],
                                               A['2d'.format(ticker)],
                                               A['3d'.format(ticker)],
                                               A['4d'.format(ticker)],
                                               A['5d'.format(ticker)],
                                               A['6d'.format(ticker)],
                                               A['7d'.format(ticker)] ))

A.head()

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,,Adj Close,Volume,1d,2d,3d,4d,5d,6d,7d,50MA,...,10MA,5MA,50STD,25STD,10STD,50UBB,25UBB,50LBB,25LBB,target
Date,Ticker,,,,,,,,,,,,,,,,,,,,,
2010-01-04,A,20.436504,3815500.0,-0.010862,-0.014377,-0.015655,-0.015975,-0.015336,-0.027157,-0.019489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2010-01-05,A,20.214516,4186000.0,-0.003553,-0.004845,-0.005168,-0.004522,-0.016473,-0.008721,0.006137,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2010-01-06,A,20.142689,3243700.0,-0.001297,-0.001621,-0.000972,-0.012966,-0.005187,0.009725,-0.013614,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2010-01-07,A,20.116571,3095100.0,-0.000325,0.000325,-0.011685,-0.003895,0.011036,-0.012334,-0.000325,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2010-01-08,A,20.110041,3733900.0,0.000649,-0.011364,-0.003572,0.011364,-0.012013,0.000000,-0.005520,NaN,...,NaN,20.204064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
